# Getting Geographies for Analysis

Analysis frequently starts with identifying a geographic area for analysis. Many times this area is a standard geography such as a Core Based Statistica Area (CBSA) or a slightly larger Designated Market Area (DMA) around major metropolitan areas in the United States. Since the Business Analyst data is organized in countries, these can be discovered by interrogating the country of interest, and selecting the area of interest starts with selecting the country for analysis. Within this area of interest, many times smaller levels of geography are used for subsequent data aggregation and analysis, and by using a few available methods, smaller geographies can be selecting for subsequent analysis as well. Ultimately, getting block groups for an area of analysis such as Seattle should be as easy as programatically saying, "_I want block groups in Seattle._"

The first step is importing necessary modules. Long term, the idea is this will reside in the ArcGIS Python API, so instead of all the stuff in the cell below, getting started will be as easy as... 


```
from arcgis.dm import Country
from arcgis.dm.utils import get_countries
```

In [1]:
from dm import Country
from dm.utils import get_countries

## Get a Country

The first step is simply discovering what countries are available for analysis.

In [2]:
get_countries()

,geo_ref,country,year
0,CAN_ESRI_2019,CAN,2019
1,USA_ESRI_2020,USA,2020


My machine has both data for the United States and Canada installed, so these are listed here. Using the three letter identifier from the `country` column, I can instantiate a Country object instance to work with.

In [3]:
usa = Country('USA')

usa

<dm.Country - USA (local)>

## Get an Area of Interest

With the country instantiated, next we can select our area of analysis. For this example I am going to select the Seattle CBSA. First though, I want to see what geographic levels are available, and know how to reference the CBSAs.

In [4]:
usa.geographies

,geo_name,geo_alias,col_id,col_name,feature_class_path
0,block_groups,Block Groups,ID,NAME,D:\arcgis\ba_data\us_2020\Data\Demographic Dat...
1,census_tracts,Census Tracts,ID,NAME,D:\arcgis\ba_data\us_2020\Data\Demographic Dat...
2,cities_and_towns_places,Cities and Towns (Places),ID,NAME,D:\arcgis\ba_data\us_2020\Data\Demographic Dat...
3,zip_codes,ZIP Codes,ID,NAME,D:\arcgis\ba_data\us_2020\Data\Demographic Dat...
4,county_subdivisions,County Subdivisions,ID,NAME,D:\arcgis\ba_data\us_2020\Data\Demographic Dat...
5,counties,Counties,ID,NAME,D:\arcgis\ba_data\us_2020\Data\Demographic Dat...
6,cbsas,CBSAs,ID,NAME,D:\arcgis\ba_data\us_2020\Data\Demographic Dat...
7,congressional_districts,Congressional Districts,ID,NAME,D:\arcgis\ba_data\us_2020\Data\Demographic Dat...
8,dmas,DMAs,ID,NAME,D:\arcgis\ba_data\us_2020\Data\Demographic Dat...
9,states,States,ID,NAME,D:\arcgis\ba_data\us_2020\Data\Demographic Dat...


The above geographies are listed heirarchially from smallest to largest area. While we later are going to retrieve the lowest geographic level, we first need to retrieve the Seattle CBSA as our area of interest. Before doing this, we can check to see what is available using a much faster method.

In [5]:
usa.cbsas.get_names('seattle')

0    Seattle-Tacoma-Bellevue, WA Metropolitan Stati...
Name: geo_name, dtype: object

Since this only returns one record, we can easily use this query string to get a Spatially Enabled DataFrame for the Seattle CBSA.

In [6]:
cbsa_df = usa.cbsas.get('seattle')

cbsa_df

,ID,NAME,SHAPE
0,42660,"Seattle-Tacoma-Bellevue, WA Metropolitan Stati...","{""rings"": [[[-122.62951999978839, 47.163890001..."


Since a Spatially Enabled DataFrame, it is not difficult to display the results on a map to ensure it is the area we want to work with. In this case I take the extra step of setting the basemap since I like the dark gray.

NOTE: This will not display unless actively running in an environment with the web mapping widget configured.

In [7]:
wm01 = cbsa_df.spatial.plot()
wm01.basemap = 'gray-vector'
wm01

MapView(layout=Layout(height='400px', width='100%'))

## Get Geographies for Analysis

The area of interest can now be used to retrieve the block groups for subsequent analysis.

In [8]:
bg_df = cbsa_df.dm.block_groups.get()

bg_df.head()

,ID,NAME,SHAPE
0,530530701003,530530701.003,"{""rings"": [[[-122.1652430000312, 47.0830489997..."
1,530530714071,530530714.071,"{""rings"": [[[-122.33136200011126, 47.064063999..."
2,530530714072,530530714.072,"{""rings"": [[[-122.35775599975194, 47.065219999..."
3,530530714073,530530714.073,"{""rings"": [[[-122.36863199991484, 47.053068000..."
4,530530714112,530530714.112,"{""rings"": [[[-122.4110792494156, 47.0717044998..."


Frequently, we simply want to access the lowest level of geography without having to explicitly specify the name.

In [9]:
lvl_df = cbsa_df.dm.level(0).get()

lvl_df.head()

,ID,NAME,SHAPE
0,530530701003,530530701.003,"{""rings"": [[[-122.1652430000312, 47.0830489997..."
1,530530714071,530530714.071,"{""rings"": [[[-122.33136200011126, 47.064063999..."
2,530530714072,530530714.072,"{""rings"": [[[-122.35775599975194, 47.065219999..."
3,530530714073,530530714.073,"{""rings"": [[[-122.36863199991484, 47.053068000..."
4,530530714112,530530714.112,"{""rings"": [[[-122.4110792494156, 47.0717044998..."


There also is the case when we need to select an area of interest, which is not a standard geography level, just using a geometry. This can be accomplished using a geometry object.

In this instance, although I am using the geometry from the CBSA, it is only an example. Any polygon geometry can be used.

In [10]:
aoi_geom = cbsa_df.iloc[0]['SHAPE']

wthn_df = usa.level(0).within(aoi_geom)

wthn_df.head()

,ID,NAME,SHAPE
0,530530701003,530530701.003,"{""rings"": [[[-122.1652430000312, 47.0830489997..."
1,530530714071,530530714.071,"{""rings"": [[[-122.33136200011126, 47.064063999..."
2,530530714072,530530714.072,"{""rings"": [[[-122.35775599975194, 47.065219999..."
3,530530714073,530530714.073,"{""rings"": [[[-122.36863199991484, 47.053068000..."
4,530530714112,530530714.112,"{""rings"": [[[-122.4110792494156, 47.0717044998..."


## Function Chaining for Efficiency

Once familiar with the workflow, the entire workflow can be chained for efficiency. This enables functionally being able to say, "_I want all block groups, or simply the lowest level of geography, in Seattle._"

In [11]:
sea_bg_df = usa.cbsas.get('seattle').dm.level(0).get()

sea_bg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   ID      2478 non-null   object  
 1   NAME    2478 non-null   object  
 2   SHAPE   2478 non-null   geometry
dtypes: geometry(1), object(2)
memory usage: 58.2+ KB


In this way, moving to a new area of interest is as simple as switching the selector to identify the name of the area to search for.

In [12]:
pdx_bg_df = usa.cbsas.get('portland').dm.level(0).get()

pdx_bg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1792 entries, 0 to 1791
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   ID      1792 non-null   object  
 1   NAME    1792 non-null   object  
 2   SHAPE   1792 non-null   geometry
dtypes: geometry(1), object(2)
memory usage: 42.1+ KB


Using these methods makes it realatively straightforward to be able to quickly get relevant geographies for an area of interest - to be able to programatically say, "_I want the smallest geographies for a metro area,_" even if in a completely different country.

In [13]:
can = Country('CAN')

van_lvl_df = can.cmacas.get('vancouver').dm.level(0).get()

van_lvl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3442 entries, 0 to 3441
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   ID      3442 non-null   object  
 1   NAME    3442 non-null   object  
 2   SHAPE   3442 non-null   geometry
dtypes: geometry(1), object(2)
memory usage: 80.8+ KB


In [14]:
van_lvl_df.head()

,ID,NAME,SHAPE
0,59150369,59150369,"{""rings"": [[[-123.0705300003746, 49.2525500003..."
1,59150370,59150370,"{""rings"": [[[-123.07615000003094, 49.251719999..."
2,59150373,59150373,"{""rings"": [[[-123.08187000014466, 49.250950000..."
3,59150375,59150375,"{""rings"": [[[-123.08198000064783, 49.247739999..."
4,59150376,59150376,"{""rings"": [[[-123.08268000025659, 49.246560000..."


NOTE: Just as above, unless actively running in an environment with the web mapping widget configured, the mapping widget being created below will not display.

In [15]:
wm02 = van_lvl_df.spatial.plot()
wm02.basemap = 'gray-vector'
wm02

MapView(layout=Layout(height='400px', width='100%'))

Finally, since a Spatially Enabled DataFrame, saving to a variety of formats, including an Esri Feature Class, is quite straightforward.

In [16]:
van_lvl_df.spatial.to_featureclass('../data/interim/interim.gdb/vancouver_lvl00')

'D:\\projects\\demographic-modeling-module\\data\\interim\\interim.gdb\\vancouver_lvl00'